# Class and Records
What is a record?
- Indroduced in C# 9
- reference type for encapsulating immutable types 
    - comes from functional programming where immutablilty is king
        - data is kept seperate from logic in immutable types called records
- C# 10 introduces record struct's

What is the motivation for having a record?
- Immutable by default
    - concise syntax for creating immutable properties
- Value semantics
    - variables are equal based on value, rather than reference
    - this is often what you want
        - but is non trivial to achieve with regular reference type

In [157]:
class Person
{
    public int Age {get; set;}
    public string Name {get; set;}

    public Person() { }

    public Person(int age, string name)
    {
        Age = age;
        Name = name;
    }
}

In [158]:
var q = new Person(1, "Quinten");
var q1 = new Person 
{
    Age = 1, 
    Name = "Quinten"
};
Console.WriteLine($"Person {{Age = {q.Age}}}, Name = {{{q.Name}}}");
Console.WriteLine($"Person {{Age = {q1.Age}}}, Name = {{{q1.Name}}}");

// but this is false even though we are tryint to represent the same Person.
Console.WriteLine($"{q == q1}");

// and this is mutable :(
q.Name = "Bob";
Console.WriteLine($"Person {{Age = {q.Age}}}, Name = {{{q.Name}}}");

Person {Age = 1}, Name = {Quinten}
Person {Age = 1}, Name = {Quinten}
False
Person {Age = 1}, Name = {Bob}


In [159]:
class Person
{
    public int Age {get; private set;}
    public string Name {get; private set;}

    public Person() { }

    public Person(int age, string name)
    {
        Age = age;
        Name = name;
    }
}

In [160]:
var q = new Person(1, "Quinten");
var q1 = new Person(1, "Quinten");
Console.WriteLine($"Person {{Age = {q.Age}}}, Name = {{{q.Name}}}");
Console.WriteLine($"Person {{Age = {q1.Age}}}, Name = {{{q1.Name}}}");
Console.WriteLine($"{q == q1}");

Person {Age = 1}, Name = {Quinten}
Person {Age = 1}, Name = {Quinten}
False


We could implement IEquatable and all the operators... or

In [161]:
record Person(int Age, string Name);

var q = new Person(1, "Quinten");
var q1 = new Person(1, "Quinten");
Console.WriteLine(q);
Console.WriteLine(q1);
Console.WriteLine($"{q == q1}");

Person { Age = 1, Name = Quinten }
Person { Age = 1, Name = Quinten }
True


### But I like object initialization syntax
The `init` keyword allows you to create an immutable property that can be `initialized` during contruction.

In [162]:
record Person()
{
    public int Age {get; init;}
    public string Name {get; init;}
}

var q = new Person
{
    Age = 1, 
    Name = "Quinten"
};
var q1 = new Person
{
    Age = 1, 
    Name = "Quinten"
};
Console.WriteLine(q);
Console.WriteLine(q1);
Console.WriteLine($"{q == q1}");

Person { Age = 1, Name = Quinten }
Person { Age = 1, Name = Quinten }
True


In [163]:
// and immutablity is maintained
q.Name = "Bob";

Error: (2,1): error CS8852: Init-only property or indexer 'Person.Name' can only be assigned in an object initializer, or on 'this' or 'base' in an instance constructor or an 'init' accessor.

### What else does this enable?
Nondestructive mutation
- The result of a with expression is a shallow copy

In [164]:
var bob = q with {Name = "Bob"};
Console.WriteLine(q); // we still have q :)
Console.WriteLine(bob); // which means we have timetravel
Console.WriteLine(q == bob) // and at this point, they are no longer the same person

Person { Age = 1, Name = Quinten }
Person { Age = 1, Name = Bob }
False


^ did you notice the built in formatting? ^

# Introduction to Pattern Matching

Pattern matching in C# lets you test expressions for certain characteristics and take actions based on the first matching pattern. It can help you check for null values, types, and enum values more concisely and safely.

`is`

In [165]:
class Circle {
    public float Diameter => 1;
}

Circle circle;

Console.WriteLine(circle.Diameter);

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#166.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

pattern matching to save the day

In [166]:
Console.WriteLine(circle is Circle c ? c.Diameter : 0f); 
Console.WriteLine(circle is not null ? circle.Diameter : 0f); 

0
0


You should prefer pattern matching with `is` wherever possible, especially for `null` checks.
This is because pattern matching is more efficient than equality checks.
- there is no risk of overridden behavior
- overloads do not need to run
- it only checks exactly what is asked

`switch` (expression)

consice syntax for branching expression logic based on the shape of the data

In [167]:
string WaterState(int tempInFahrenheit) =>
    tempInFahrenheit switch
    {
        (> 32) and (< 212) => "liquid",
        < 32 => "solid",
        > 212 => "gas",
        32 => "solid/liquid transition",
        212 => "liquid / gas transition",
    };

Console.WriteLine(WaterState(1));
Console.WriteLine(WaterState(33));
Console.WriteLine(WaterState(220));

solid
liquid
gas


In [168]:
public record Order(int Items, decimal Cost);

public decimal CalculateDiscount(Order order) =>
    order switch
    {
        { Items: > 10, Cost: > 1000.00m } => 0.10m,
        { Items: > 5, Cost: > 500.00m } => 0.05m,
        { Cost: > 250.00m } => 0.02m,
        null => throw new ArgumentNullException(nameof(order), "Can't calculate discount on null order"),
        var someObject => 0m,
    };

var orders = new List<Order> { new Order(1, 100m), new Order(20,1000m) };
orders
    .Select(CalculateDiscount)
    .ToList()
    .ForEach(Console.WriteLine);

0
0.05


In [169]:
var input = """
04-01-2020,DEPOSIT,Initialdeposit,2250.00
04-15-2020,DEPOSIT,Refund,125.65
04-18-2020,DEPOSIT,Paycheck,825.65
04-22-2020,WITHDRAWAL,Debit,Groceries,255.73
05-01-2020,WITHDRAWAL,#1102,Rent,apt,2100.00
05-02-2020,INTEREST,0.65
05-07-2020,WITHDRAWAL,Debit,Movies,12.57
04-15-2020,FEE,5.55
""";

// bonus feature, _ the discard
decimal ParseTransaction(string[] transaction) => 
    transaction switch
    {
        [_, "DEPOSIT", _, var amount]     => decimal.Parse(amount),
        [_, "WITHDRAWAL", .., var amount] => -decimal.Parse(amount),
        [_, "INTEREST", var amount]       => decimal.Parse(amount),
        [_, "FEE", var fee]               => -decimal.Parse(fee),
        _                                 => throw new InvalidOperationException($"Record {string.Join(", ", transaction)} is not in the expected format!"),
    };

var records = 
    from row in input.Split('\n')
    select row.Split(',');

decimal balance = 0m;
foreach (string[] transaction in records)
{
    balance += ParseTransaction(transaction);
    Console.WriteLine($"Record: {string.Join(", ", transaction)}, New balance: {balance:C}");
}

, New balance: $2,250.00SIT, Initialdeposit, 2250.00
, New balance: $2,375.65SIT, Refund, 125.65
, New balance: $3,201.30SIT, Paycheck, 825.65
, New balance: $2,945.57DRAWAL, Debit, Groceries, 255.73
, New balance: $845.57THDRAWAL, #1102, Rent, apt, 2100.00
, New balance: $846.22TEREST, 0.65
, New balance: $833.65THDRAWAL, Debit, Movies, 12.57
Record: 04-15-2020, FEE, 5.55, New balance: $828.10
